## Árvore B 

O código abaixo ilustra a inclusão, busca, exclusão e percursos em árvores B, com grau máximo 2*t= 4, (t=2).

Árvores B são estruturas usadas para criação de índices em memória secundária. O tamanho dos nós é geralmente o tamanho de uma página na memória secundária, ou múltiplos do tamanho da página, com o objetivo de minimizar o número de operações de leitura em disco.

Para permitir testes online das operações de inclusão/exclusão, esta versão não possui as operações de acesso ao disco.

In [1]:
#include <iostream>
#include <stdlib.h> 
using namespace std;

#define GRAU_MAXIMO 4
#define GRAU_MINIMO 2

Estrutura para uma árvore B simples. A árvore tem um conjunto de chaves e de nós filhos. O número de nós filhos é sempre 1 unidade maior que o número de chaves.

In [2]:
struct tNo {
  int chave [GRAU_MAXIMO-1];
  tNo *p [GRAU_MAXIMO];
  int tam; 
} 

Função para inicialização do nó, com alocação de memória e atribuição de valores NULL para os ponteiros e 0 para o tamanho.

In [3]:
tNo *criaNo (){
    tNo *n = (tNo *)malloc (sizeof (tNo));
    n->tam = 0;
    for (int i=0; i < GRAU_MAXIMO ;i++)
       n->p[i] = NULL;
    return n;
}

Função que lê um token, separado por espaço, e converte para um número inteiro.

In [4]:
int token_to_num(const char *str, int *indice){    
    char token[100];
    int i = 0;
    while (str[*indice] != '\0' && str[*indice] != ' '){
        token[i] = str[*indice];
        i++;
        (*indice)++;
    }
    token[i] = '\0';
    (*indice)++;    
    return atoi(token);
}

Percurso da árvore em **PRÉ-ORDEM** em uma árvore B, pela chave.

In [5]:
void preordem(tNo *no){
   if (no != NULL) {
       cout << "(";
       for (int i=0; i < no->tam; i++){
           cout << no->chave[i] << '.';
           preordem(no->p[i]);
       }
       preordem(no->p[no->tam]);
       cout << ")";
   }
}


Percurso da árvore em **ORDEM** pela chave.

In [6]:
void emordem(tNo *no){    
   if (no != NULL) {
       for (int i=0; i < no->tam; i++){
           emordem(no->p[i]);
           cout << no->chave[i] << '.';
       }
       emordem(no->p[no->tam]);
   }
}

Percurso da árvore em **PÓS-ORDEM** pela chave.

In [7]:
void posordem(tNo * no){
   if (no != NULL) {
       for (int i=0; i < no->tam; i++){
           posordem(no->p[i]);
           if (i-1 >=0)
               cout << no->chave[i-1] << '.';
       }       
       posordem(no->p[no->tam]);     
       cout << no->chave[no->tam-1] << '.';
   }
}

Conta o número de nós da árvore.

In [8]:
int contaNos(tNo *no){
   int nos = 0; 
   if (no != NULL) {
       for (int i=0; i < no->tam; i++){           
           nos += contaNos(no->p[i]);
       }
       nos += contaNos(no->p[no->tam]);
       nos += 1;
   }
   return nos;
}

Uma árvore B pode ter mais de 1 chave por nó. Esta função conta o total de todas as chaves em todos os nós.

In [9]:
int contaChaves(tNo *no){
   int chaves = 0; 
   if (no != NULL) {
       for (int i=0; i < no->tam; i++){           
           chaves += contaChaves(no->p[i]) + 1;
       }
       chaves += contaChaves(no->p[no->tam]);
   }
   return chaves;
}

Calcula a altura da árvore. Como a árvore B é sempre balanceada, a contagem da altura é feita descendo pelo primeiro ponteiro à esquerda até o nó folha.

In [10]:
int altura (tNo *p) {
    int i = 0;
    while (p->p[0] != NULL){
        p = p->p[0];
        i ++;
    }
    return i;
}

Função que divide um nó de uma árvore B, usada na inclusão, quando o nó está cheio. Um nó está cheio quando o número de chaves é igual ao grau máximo - 1.

In [11]:
void splitNo (tNo *pai, int indice, tNo *no) {
    tNo *novo = criaNo();
    novo->tam = GRAU_MINIMO - 1;
    //atribui chaves da segunda metade para o novo nó
    for (int i=0; i < GRAU_MINIMO-1; i++){
        novo->chave[i] = no->chave[i+GRAU_MINIMO];
    }
    //atribui os pointeiros da segunda metade, se não é folha
    if (no->p[0]  != NULL) {//novo->chave[i]
        for (int i=0; i <= GRAU_MINIMO-1; i++)
            novo->p[i] = no->p[i + GRAU_MINIMO];
    }
    no->tam = GRAU_MINIMO - 1;
    //desloca os ponteiros do nó pai para a direita
    for (int i = pai->tam;  i > indice; i--)
        pai->p[i+1] = pai->p[i];
    //ultimo indice aponta para o novo nó
    pai->p[indice+1] = novo;
    //desloca as chaves do pai para a direita
    for (int i = pai->tam-1; i >= indice; i--)
        pai->chave[i+1] = pai->chave[i];
    //sobe a chave
    pai->chave[indice] = no->chave[GRAU_MINIMO-1];
    pai->tam = pai->tam +1;
}

Função que faz o split da árvore B na raiz. É diferente do split dos demais nós, pois são criados 2 novos nós: a nova raiz e o novo nó à direita.

In [12]:
tNo *splitRaiz(tNo * raiz){
    tNo *novaRaiz = criaNo();
    splitNo(novaRaiz, 0, raiz);
    novaRaiz->p[0]=raiz;
    novaRaiz->p[0]->tam = GRAU_MINIMO - 1;
    return novaRaiz;
}

Funcão para inclusão de novo nó na árvore B, dado um nó raiz. O split é feito na descida, isto é, sempre que encontrar um nó cheio, irá chamar a função __splitNo__, e continuará a descida.

In [13]:
tNo* inclui (tNo *no, int chave, tNo* raiz){
    int i = no->tam -1;
    tNo *filho;
    if (no->tam == GRAU_MAXIMO - 1 && no == raiz) {//split na raiz, tratamento específico
        raiz = splitRaiz(raiz);
        no = raiz;
        i = no->tam - 1;
    }   
    if (no->p[0] == NULL){ //encontrou um nó folha para inclusão
        while (i >= 0 && chave < no->chave[i]) {
            no->chave[i+1] = no->chave[i];
            i--;
        }
        no->chave[i+1] = chave;
        no->tam = no->tam+1;
    } 
    else { //desce na árvore e faz o split se necessário
        while (i >= 0 && chave < no->chave[i])
            i--;
        i++;
        filho = no->p[i];
        if (filho->tam == GRAU_MAXIMO - 1){
            splitNo(no, i, filho);
            if (chave > no->chave[i]) 
                i ++;
        }
        inclui( no->p[i], chave, raiz);    
    }
    return raiz;
}

Função que lê a string de entrada com as chaves, e chama a função de inclusão. Retorna a raiz da árvore.

In [14]:
tNo* montaarvore(const char *str){
    tNo *raiz = NULL;
    int i = 0, v =0;
    raiz = criaNo();
    while (str[i]!='\0')
        raiz = inclui (raiz, token_to_num(str, &i), raiz);
    return raiz;        
}


Busca em árvore B. Retorna o nó com a chave encontrada.

In [15]:
tNo *busca (tNo *no, int chave) {
    if( no != NULL ){ 
        for(int i=0; i < no->tam; i++) {
            if( no->chave[i] == chave ) 
                return no; 
            else 
                if( chave < no->chave[i] ) 
                    return busca( no->p [i], chave ); 
        }
        return busca( no->p[ no->tam ], chave ); 
    }
    else
        return NULL;
}

Função auxiliar para retornar o índice da chave em um nó.

In [16]:
int indiceChave(tNo *no, int chave){
    int i;
    for (i = 0; i < no->tam && no->chave[i] != chave; i++);
    return i;
}

Dado um nó, encontra o índice da sub-árvore onde se encontra a chave.

In [17]:
int posicaoSubArvore(tNo *no, int chave){
    int i = no->tam -1;
    while (i >= 0 && chave < no->chave[i])
        i--;
    i++;    
    return i;
}

Desloca todas as chaves e os ponteiros de um nó 1 posição para a direita.

In [18]:
void deslocaDireita(tNo *no){
    int i;
    for (i=no->tam ; i > 0 ; i--){
        no->chave[i] = no->chave[i-1];
        no->p[i+1] = no->p[i];
    }
    no->p[i+1] = no->p[i];
}

Desloca as chaves e os ponteiros de um nó 1 posição para a esquerda, até um índice determinado.

In [19]:
void deslocaEsquerda(tNo *no, int indice){
    int i;
    for (i=indice ; i < no->tam ; i++){
        no->chave[i] = no->chave[i+1];
        no->p[i] = no->p[i+1];
    }
    no->p[i] = no->p[i+1];
}

Fusiona 2 nós passados como parâmetro, e retorna o nó fusionado. Fusiona o nó direito no nó esquerdo.
Ajusta as chaves e os ponteiros do nó, e libera o nó direito da memória.

In [20]:
tNo* fusionaNo(tNo *esq, int chave, tNo * dir){
    int tamEsq = esq->tam;
    esq->chave[tamEsq] = chave;
    int i;
    tamEsq++;
    esq->tam ++;
    for (i=0; i < dir->tam; i++){//copia chaves e ponteiros da direita para no esquerdo
        esq->chave[tamEsq + i] = dir->chave[i];
        esq->p[tamEsq + i] = dir->p[i];
    }
    esq->p[tamEsq + i] = dir->p[i];
    esq->tam = esq->tam + dir->tam;
    free(dir);
    return esq;
}

Função de exclusão em árore B. Possui 3 casos:
1. Chave está no nó e é folha
2. Chave está nó e é interno
    - se filho esquerdo tem ao menos t chaves 
    - se filho direito tem ao menos t chaves
    - se filhos direito ou esquerdo não tenham ao menos t chaves
3. Chave não está no nó interno
    - se a raiz da subárvore tem t-1 chaves mas irmão imediato (esquerdo ou direito) tem ao menos t chaves
    - se irmãos imediatos possuem t-1 chaves


In [21]:
tNo* exclui (tNo *no, int chave, tNo* raiz){
    int j, i = no->tam -1;
    int aux=0, excluiuCaso2 = 0;
    tNo *filho, *irmao, *sucessor, *antecessor;
    if (indiceChave(no, chave) == 0 && no->tam == 1 && no == raiz) {//split na raiz
        cout << "\nExcluiu nó raiz";
        free(no);
        return NULL;
    }   
    if (no->p[0] == NULL){ //Caso 1: chave está no nó e é folha
        cout << "\nCaso 1: nó folha";
        //encontra o indice da chave
        j = indiceChave(no, chave);
        if (j == no->tam) return NULL; //não encontrou a chave
        while (j < no->tam) { 
            no->chave[j] = no->chave[j+1];
            j++;
        }
        no->tam = no->tam-1;
    } 
    else { //desce na árvore e faz o merge se necessário
        j = indiceChave(no, chave);
        i = posicaoSubArvore(no, chave);        
        if (j < no->tam) {
            //Caso 2: encontrou a chave em nó interno
            if (no->p[j] != NULL && no->p[j]->tam >= GRAU_MINIMO) {
                cout << "\nCaso 2.antecessor";
                antecessor = no->p[j];
                while (antecessor->p[antecessor->tam - 1] != NULL)
                    antecessor = antecessor->p[antecessor->tam - 1];
                no->chave[j] = antecessor->chave[antecessor->tam -1];
                exclui(no->p[j], antecessor->chave[antecessor->tam-1],raiz);
            } else {
                if (no->p[j+1] != NULL && no->p[j+1]->tam >= GRAU_MINIMO) {
                    cout << "\nCaso 2.sucessor";
                    sucessor = no->p[j+1];
                    while (sucessor->p[0] != NULL)
                        sucessor = sucessor->p[0];
                    no->chave[j] = sucessor->chave[0];
                    exclui(sucessor, sucessor->chave[0], raiz);
                    }
                else {
                    cout << "\nCaso 2.sem grau mínimo";
                    filho = fusionaNo(no->p[j], no->chave[j], no->p[j+1]);
                    no->chave[j] = no->chave[j+1];
                    deslocaEsquerda(no, j+1);
                    no->tam --;
                    exclui(filho, chave, raiz);              
                }
            } 
        }
        else { //Caso 3: não encontrou a chave em nó interno
            filho = no->p[i]; //sub arvore da chave
            if (no->p[i]->tam == GRAU_MINIMO - 1){
                if ( (i-1)>=0 && no->p[i-1]->tam >= GRAU_MINIMO){
                    cout << "\nCaso 3.a.esq)"; //filho esquerdo com ao menos t chaves
                    irmao = no->p[i-1];
                    aux = no->chave[i-1]; 
                    irmao->tam --;
                    no->chave[i-1] = irmao->chave[irmao->tam];
                    deslocaDireita(filho);
                    filho->tam ++;
                    filho->chave[0] = aux;
                    filho->p[0] = irmao->p[irmao->tam+1];
                }
                else {
                    if ((i+1) <= no->tam && no->p[i+1]->tam >= GRAU_MINIMO){
                        cout << "\nCaso 3.a.dir)"; //filho direito com ao menos t chaves
                        irmao = no->p[i+1];
                        aux = no->chave[i]; 
                        irmao->tam --;
                        no->chave[i] = irmao->chave[0];
                        deslocaEsquerda(irmao, 0);
                        filho->tam ++;
                        filho->chave[filho->tam-1] = aux;
                        filho->p[filho->tam] = irmao->p[0];                        
                    }
                    else { //fusao de nós
                        cout << "\nCaso 3.b)";
                        if (i == 0){//primeiro nó, fusiona com o direito
                            filho = fusionaNo(filho, no->chave[0], no->p[i+1]);
                            no->chave[0] = no->chave[1];
                            deslocaEsquerda(no, i+1);
                        } else { //fusiona com o esquerdo
                            filho = fusionaNo(no->p[i-1], no->chave[i-1], filho);
                            no->chave[i-1] = no->chave[i];
                            deslocaEsquerda(no, i);
                            i--;
                        }
                        no->tam --;
                    }   
                }
                
            }
        exclui( no->p[i], chave, raiz);
        }
    }
    return raiz;
}


In [22]:
void iniciaprograma(){
    tNo *raiz = NULL;
    
    //char entrada[] = "20 13 17 22 11 23 12 9 25 21 18 19\0";
    char entrada[] = "10 2 12 20 25 30\0";
    raiz=montaarvore(entrada); 
    
    cout << "\nPercurso em pré-ordem: ";
    preordem(raiz); cout << "\n";
    
    exclui(raiz, 30, raiz);
    exclui(raiz, 10, raiz);
    
    cout << endl << "\nPré-ordem: "; preordem(raiz); 
    cout << endl << "Total de nós " << contaNos(raiz);
    cout << endl << "Total de chaves " << contaChaves(raiz);  
    
}

In [23]:
iniciaprograma();


Percurso em pré-ordem: (10.(2.)20.(12.)(25.30.))

Caso 1: nó folha
Caso 2.sem grau mínimo
Caso 1: nó folha

Pré-ordem: (20.(2.12.)(25.))
Total de nós 3
Total de chaves 4